# Librerias

In [1]:
import pandas as pd
import numpy as np
import unicodedata
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix
)
import seaborn as sns
import matplotlib.pyplot as plt
import unicodedata
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from category_encoders import TargetEncoder

# Datos

In [7]:
df=pd.read_csv(r'C:\Users\Usuario\Documents\Trabajo Ingenieria Modelos\Bases de Datos\train.csv', encoding="utf-8")
df.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


# Preprocesamiento General

In [ ]:
if df['F_TIENEINTERNET'].equals(df['F_TIENEINTERNET.1']):
    df = df.drop(columns=['F_TIENEINTERNET.1'])
    print("Se elimino la variable F_TIENEINTERNET.1")

# Moda global para variables categóricas simples
variables_imputar = ['F_ESTRATOVIVIENDA', 'F_EDUCACIONMADRE', 'F_EDUCACIONPADRE']
for col in variables_imputar:
    print(f"\n📊 Variable: {col}")
    nulos_antes = df[col].isna().sum()
    print(f" - Valores nulos antes de imputar: {nulos_antes}")
    moda_valor = df[col].mode()[0]
    print(f" - Valor usado para imputar (moda): {moda_valor}")
    df.loc[:, col] = df[col].fillna(moda_valor)
    nulos_despues = df[col].isna().sum()
    print(f" - Valores nulos después de imputar: {nulos_despues}")
print("\n✅ Imputación completada correctamente.")

# Imputación por grupos para variables categóricas
imputaciones_grupales = {
    'F_TIENEAUTOMOVIL': 'F_ESTRATOVIVIENDA',
    'F_TIENELAVADORA': 'F_ESTRATOVIVIENDA',
    'F_TIENEINTERNET': 'F_ESTRATOVIVIENDA',
    'F_TIENECOMPUTADOR': 'F_TIENEINTERNET',
    'E_PAGOMATRICULAPROPIO': 'F_ESTRATOVIVIENDA',
    'E_HORASSEMANATRABAJA': 'F_ESTRATOVIVIENDA',
    'E_VALORMATRICULAUNIVERSIDAD': 'F_ESTRATOVIVIENDA'
}

for var, grupo in imputaciones_grupales.items():
    print(f"\n📊 Variable: {var} (agrupada por {grupo})")
    nulos_antes = df[var].isna().sum()
    print(f" - Valores nulos antes de imputar: {nulos_antes}")
    
    # Imputación por moda dentro del grupo
    df[var] = df.groupby(grupo)[var].transform(
        lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'No')
    )
    
    nulos_despues = df[var].isna().sum()
    print(f" - Valores nulos después de imputar: {nulos_despues}")

print("\n✅ Imputación por grupos completada correctamente.")

# Convertir PERIODO_ACADEMICO a categórica
df["PERIODO_ACADEMICO"] = df["PERIODO_ACADEMICO"].astype(str)
df["PERIODO_ACADEMICO"] = df["PERIODO_ACADEMICO"].astype("category")

def quitar_tildes(texto):
    if isinstance(texto, str):
        texto = unicodedata.normalize("NFD", texto)
        texto = "".join(c for c in texto if unicodedata.category(c) != "Mn")
        return texto.upper()  
    return texto

df["E_PRGM_ACADEMICO"] = df["E_PRGM_ACADEMICO"].apply(quitar_tildes)
print("Homogeneización Realizada")

Se elimino la variable F_TIENEINTERNET.1


In [19]:
import pandas as pd
import unicodedata

def preprocesar_datos(df):
    df = df.copy()

    # -------------------------------------
    # 1. Eliminar duplicado F_TIENEINTERNET.1
    # -------------------------------------
    if "F_TIENEINTERNET.1" in df.columns:
        if df["F_TIENEINTERNET"].equals(df["F_TIENEINTERNET.1"]):
            df = df.drop(columns=["F_TIENEINTERNET.1"])
            print("🧹 Se eliminó la variable duplicada: F_TIENEINTERNET.1")

    # -------------------------------------
    # 2. Imputación de moda global
    # -------------------------------------
    variables_imputar = [
        "F_ESTRATOVIVIENDA",
        "F_EDUCACIONMADRE",
        "F_EDUCACIONPADRE"
    ]

    for col in variables_imputar:
        if col in df.columns:
            moda = df[col].mode()[0]
            df[col] = df[col].fillna(moda)

    print("📌 Imputación global completada")

    # -------------------------------------
    # 3. Imputación por grupos (estrato, internet)
    # -------------------------------------
    imputaciones_grupales = {
        'F_TIENEAUTOMOVIL': 'F_ESTRATOVIVIENDA',
        'F_TIENELAVADORA': 'F_ESTRATOVIVIENDA',
        'F_TIENEINTERNET': 'F_ESTRATOVIVIENDA',
        'F_TIENECOMPUTADOR': 'F_TIENEINTERNET',
        'E_PAGOMATRICULAPROPIO': 'F_ESTRATOVIVIENDA',
        'E_HORASSEMANATRABAJA': 'F_ESTRATOVIVIENDA',
        'E_VALORMATRICULAUNIVERSIDAD': 'F_ESTRATOVIVIENDA'
    }

    for col, grupo in imputaciones_grupales.items():
        if col in df.columns and grupo in df.columns:

            df[col] = df.groupby(grupo)[col].transform(
                lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "No")
            )

    print("📌 Imputación por grupos completada")

    # -------------------------------------
    # 4. Convertir PERIODO_ACADEMICO a categoría
    # -------------------------------------
    if "PERIODO_ACADEMICO" in df.columns:
        df["PERIODO_ACADEMICO"] = df["PERIODO_ACADEMICO"].astype(str).astype("category")

    # -------------------------------------
    # 5. Función para quitar tildes
    # -------------------------------------
    def quitar_tildes(texto):
        if isinstance(texto, str):
            texto = unicodedata.normalize("NFD", texto)
            texto = "".join(c for c in texto if unicodedata.category(c) != "Mn")
            return texto.upper()
        return texto

    # Aplicar a PRGM_ACADEMICO si existe
    if "E_PRGM_ACADEMICO" in df.columns:
        df["E_PRGM_ACADEMICO"] = df["E_PRGM_ACADEMICO"].apply(quitar_tildes)

    print("✨ Homogeneización de textos realizada")
    print("✅ Preprocesamiento completado")

    return df


# XGBoost

In [10]:
# ============================================
#   1. IMPORTS
# ============================================
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier


# ============================================
#   2. ORDENAR LAS CLASES MANUALMENTE
# ============================================

orden_clases = ["bajo", "medio-bajo", "medio-alto", "alto"]

encoder_y = OrdinalEncoder(categories=[orden_clases])


# ============================================
#   3. DEFINIR VARIABLES
# ============================================

cat_cols = [
    "F_ESTRATOVIVIENDA", "F_TIENEINTERNET", "F_EDUCACIONPADRE",
    "F_TIENELAVADORA", "F_TIENEAUTOMOVIL", "E_PRIVADO_LIBERTAD",
    "E_PAGOMATRICULAPROPIO", "F_TIENECOMPUTADOR", "F_EDUCACIONMADRE", "E_VALORMATRICULAUNIVERSIDAD",
    "E_HORASSEMANATRABAJA"
]

X = df[cat_cols].copy()
y = df["RENDIMIENTO_GLOBAL"].copy()

# Codificación ORDENADA del target
y_encoded = encoder_y.fit_transform(y.to_numpy().reshape(-1, 1)).astype(int).ravel()


# ============================================
#   4. TRAIN / TEST SPLIT
# ============================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.30, random_state=42, stratify=y_encoded
)


# ============================================
#   5. PREPROCESAMIENTO PARA XGBOOST
# ============================================

pre_xgb = ColumnTransformer([
    ("cat", Pipeline([
        ('imp', SimpleImputer(strategy="most_frequent")),
        ('enc', OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1))
    ]), cat_cols)
])


# ============================================
#   6. MODELO XGBOOST
# ============================================

xgb = XGBClassifier(
    objective="multi:softmax",
    num_class=4,
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    tree_method="hist"   # más rápido
)


pipe_xgb = Pipeline([
    ("prep", pre_xgb),
    ("model", xgb)
])


# ============================================
#   7. ENTRENAR
# ============================================

pipe_xgb.fit(X_train, y_train)


# ============================================
#   8. PREDICCIÓN
# ============================================

pred_xgb = pipe_xgb.predict(X_test)

# Decodificar predicciones y_test para ver etiquetas originales
y_test_text   = encoder_y.inverse_transform(y_test.reshape(-1,1)).ravel()
pred_text     = encoder_y.inverse_transform(pred_xgb.reshape(-1,1)).ravel()

# ============================================
#   9. EVALUACIÓN
# ============================================

print("\n===== XGBOOST ORDENADO =====\n")
print(classification_report(y_test_text, pred_text, digits=4))
cm = confusion_matrix(y_test_text, pred_text)
print("\nMatriz de confusión:\n")
print(cm)


===== XGBOOST ORDENADO =====

              precision    recall  f1-score   support

        alto     0.4812    0.6040    0.5357     52686
        bajo     0.4012    0.5243    0.4545     51896
  medio-alto     0.3021    0.2088    0.2469     51486
  medio-bajo     0.3130    0.2314    0.2661     51682

    accuracy                         0.3934    207750
   macro avg     0.3744    0.3921    0.3758    207750
weighted avg     0.3750    0.3934    0.3768    207750


Matriz de confusión:

[[31822  7052  8329  5483]
 [ 7158 27209  7026 10503]
 [16769 13704 10750 10263]
 [10380 19861  9483 11958]]
